# Customer.IO App API - Communications

This notebook demonstrates how to use the Customer.IO App API for communications including transactional emails, broadcast campaigns, and push notifications.

The App API focuses on direct message delivery and is separate from the Data Pipelines API which handles customer data management.

## Setup

First, let's import the necessary modules and set up authentication.

In [ ]:
import osimport sysfrom dotenv import load_dotenv# For local development with src imports (backwards compatibility)# Add src directory to path if it existsnotebook_dir = os.path.dirname(os.path.abspath(''))project_root = os.path.abspath(os.path.join(notebook_dir, '..', '..'))src_path = os.path.join(project_root, 'src')if os.path.exists(src_path):    sys.path.insert(0, src_path)    # Try importing from src (local development pattern)    try:        from src.app_api.auth import AppAPIAuth        from src.app_api.client import (            send_transactional,            trigger_broadcast,            send_push        )    except ImportError:        # Fall back to package imports        from app_api.auth import AppAPIAuth        from app_api.client import (            send_transactional,            trigger_broadcast,            send_push        )else:    # Databricks or package installation - use package imports    from app_api.auth import AppAPIAuth    from app_api.client import (        send_transactional,        trigger_broadcast,        send_push    )# Load environment variablesload_dotenv()print("Customer.IO App API client library imported successfully")

## Databricks Environment SetupThis section handles the installation of required libraries and configuration for Databricks clusters. Skip this section if running locally.

In [ ]:
# Databricks library installation# Uncomment the following lines when running on Databricks# %pip install customerio-api-client# OR if using a wheel file:# %pip install /dbfs/path/to/customerio_api_client-1.0.0-py3-none-any.whl# For development without packaging, install individual requirements:# %pip install httpx>=0.25.0 pydantic>=2.0.0 pandas>=2.0.0 python-dotenv>=1.0.0 structlog>=24.0.0# Restart kernel after installation if needed# import dbutils# dbutils.library.restartPython()

## Configuration

Configure your Customer.IO App API credentials. For security, credentials should be stored as environment variables rather than hardcoded in notebooks.

In [ ]:
# Configuration - Replace with your actual credentials# Recommended: Set these as environment variablesAPI_TOKEN = os.getenv('CUSTOMERIO_APP_API_TOKEN', 'your-app-api-token-here')    REGION = dbutils.secrets.get(scope="customer-io", key="region")    print("Using Databricks secrets for credentials")except:    # Fall back to environment variables    REGION = os.getenv('CUSTOMERIO_REGION', 'us')  # 'us' or 'eu'    print("Using environment variables for credentials")print(f"Configured for region: {REGION}")print(f"API token configured: {'Yes' if API_TOKEN != 'your-app-api-token-here' else 'No - using placeholder'}")

## Client Initialization

Initialize the Customer.IO App API client with your credentials and region settings.

In [ ]:
# Initialize the Customer.IO App API client
auth = AppAPIAuth(
    api_token=API_TOKEN,
    region=REGION
)

print(f"App API client initialized successfully")
print(f"Region: {auth.region}")
print(f"Base URL: {auth.base_url}")

## Transactional Email Communication

Transactional emails are triggered by specific user actions or events. They are highly personalized and time-sensitive messages like password resets, order confirmations, or welcome emails.

### Email to Direct Address

Send an email directly to an email address without requiring a customer profile:

In [ ]:
# Send transactional email to a direct email address
try:
    response = send_transactional(
        auth=auth,
        transactional_message_id=1,  # Replace with your actual message ID
        to="customer@example.com",
        message_data={
            "customer_name": "John Doe",
            "order_number": "ORD-12345",
            "order_total": "$99.99",
            "delivery_date": "2024-01-15"
        }
    )
    
    print(f"Email sent successfully!")
    print(f"Delivery ID: {response.get('delivery_id')}")
    
except Exception as e:
    print(f"Error sending email: {e}")
    # Note: This will show an error unless you have a valid transactional_message_id

### Email Using Customer Identifiers

Send an email using customer identifiers (requires the customer to exist in your workspace):

In [ ]:
# Send transactional email using customer identifiers
try:
    response = send_transactional(
        auth=auth,
        transactional_message_id=1,  # Replace with your actual message ID
        identifiers={
            "email": "customer@example.com"
            # Can also use: {"id": "customer_123"} for internal ID
        },
        message_data={
            "subject_line": "Your Order Confirmation",
            "product_name": "Customer.IO Integration Guide",
            "download_link": "https://customer.io/docs",
            "support_email": "support@example.com"
        }
    )
    
    print(f"Email sent using customer identifiers!")
    print(f"Delivery ID: {response.get('delivery_id')}")
    
except Exception as e:
    print(f"Error sending email: {e}")

## Broadcast Campaign Communication

Broadcast campaigns are one-time messages sent to a segment of customers. They are typically used for marketing announcements, product launches, or newsletters.

### Trigger API Broadcast

Trigger a pre-configured broadcast campaign via the API:

In [ ]:
# Trigger a broadcast campaign
try:
    response = trigger_broadcast(
        auth=auth,
        broadcast_id=1,  # Replace with your actual broadcast ID
        data={
            "campaign_name": "Spring Sale 2024",
            "discount_percentage": 25,
            "sale_end_date": "2024-03-31",
            "featured_products": [
                "Premium Plan",
                "Enterprise Solution"
            ]
        },
        recipients={
            "segment": {"id": 1}  # Target specific segment
        }
    )
    
    print(f"Broadcast triggered successfully!")
    print(f"Trigger ID: {response.get('id')}")
    
except Exception as e:
    print(f"Error triggering broadcast: {e}")
    # Note: This will show an error unless you have a valid broadcast_id

### Broadcast with Advanced Targeting

Trigger a broadcast with more sophisticated recipient targeting:

In [ ]:
# Trigger broadcast with advanced targeting
try:
    response = trigger_broadcast(
        auth=auth,
        broadcast_id=2,  # Replace with your actual broadcast ID
        data={
            "headline": "Exclusive Beta Access",
            "feature_name": "Advanced Analytics Dashboard",
            "beta_signup_url": "https://example.com/beta-signup",
            "expiration_date": "2024-02-28"
        },
        recipients={
            "emails": [
                "beta-user1@example.com",
                "beta-user2@example.com"
            ]
        }
    )
    
    print(f"Targeted broadcast sent!")
    print(f"Response: {response}")
    
except Exception as e:
    print(f"Error sending targeted broadcast: {e}")

## Push Notification Communication

Push notifications are real-time messages sent directly to users' mobile devices or browsers. They are excellent for urgent updates, reminders, and engagement.

### Mobile Push Notification

Send a push notification to mobile devices:

In [ ]:
# Send push notification
try:
    response = send_push(
        auth=auth,
        identifiers={
            "email": "mobile-user@example.com"
        },
        title="Order Status Update",
        message="Your order #12345 has been shipped and is on its way!",
        device_tokens=[
            "device_token_12345_ios",
            "device_token_67890_android"
        ]
    )
    
    print(f"Push notification sent successfully!")
    print(f"Delivery ID: {response.get('delivery_id')}")
    
except Exception as e:
    print(f"Error sending push notification: {e}")
    # Note: Push notifications require proper setup and valid device tokens

### Push Notification with Rich Content

Send a push notification with additional payload data:

In [ ]:
# Send rich push notification
try:
    response = send_push(
        auth=auth,
        identifiers={
            "id": "user_123"
        },
        title="New Message",
        message="You have a new message from Customer.IO",
        device_tokens=["device_token_example"],
        # Additional push notification data
        sound="notification.wav",
        badge=1,
        data={
            "deep_link": "/messages/inbox",
            "category": "social",
            "action_url": "https://app.example.com/messages"
        }
    )
    
    print(f"Rich push notification sent!")
    print(f"Response: {response}")
    
except Exception as e:
    print(f"Error sending rich push notification: {e}")

## Error Handling and Best Practices

Proper error handling is crucial when working with external APIs. Here are common error scenarios and how to handle them:

### Authentication Errors

In [ ]:
# Example of handling authentication errors
try:
    # This will demonstrate an authentication error with an invalid token
    invalid_auth = AppAPIAuth(
        api_token="invalid_token_example",
        region="us"
    )
    
    response = send_transactional(
        auth=invalid_auth,
        transactional_message_id=1,
        to="test@example.com"
    )
    
except Exception as e:
    error_message = str(e).lower()
    
    if "401" in error_message or "unauthorized" in error_message:
        print("Authentication Error: Please check your API token")
        print("- Verify your token in Customer.IO dashboard")
        print("- Ensure you're using the App API token (not Pipelines API key)")
    else:
        print(f"Other error occurred: {e}")

### Validation Errors

In [ ]:
# Example of handling validation errors
try:
    # This will demonstrate a validation error (missing required parameters)
    response = send_push(
        auth=auth,
        identifiers={"email": "test@example.com"},
        # Missing required title, message, and device_tokens
    )
    
except ValueError as e:
    print(f"Validation Error: {e}")
    print("Please ensure all required parameters are provided")
    
except Exception as e:
    print(f"API Error: {e}")

### Resource Not Found Errors

In [ ]:
# Example of handling resource not found errors
try:
    # This will demonstrate a 404 error with non-existent message ID
    response = send_transactional(
        auth=auth,
        transactional_message_id=99999999,  # Non-existent message ID
        to="test@example.com"
    )
    
except Exception as e:
    error_message = str(e).lower()
    
    if "404" in error_message or "not found" in error_message:
        print("Resource Not Found Error:")
        print("- Check that your transactional message ID exists")
        print("- Verify broadcast ID is correct")
        print("- Ensure customer exists for identifier-based sending")
    elif "403" in error_message or "forbidden" in error_message:
        print("Permission Error:")
        print("- Your API token may not have sufficient permissions")
        print("- Check your workspace settings")
    else:
        print(f"Other error occurred: {e}")

## Summary

This notebook demonstrated the three main communication channels available through the Customer.IO App API:

1. **Transactional Emails**: Personalized, triggered emails for user actions
2. **Broadcast Campaigns**: Marketing messages sent to customer segments
3. **Push Notifications**: Real-time mobile and browser notifications

### Key Points:

- **Authentication**: Use Bearer token authentication with AppAPIAuth
- **Rate Limiting**: The client automatically handles API rate limits
- **Error Handling**: Always wrap API calls in try-catch blocks
- **Test Data**: Use valid message IDs and broadcast IDs from your Customer.IO workspace
- **Direct vs Identifier**: You can send to email addresses directly or use customer identifiers

### Next Steps:

1. Set up your transactional messages and broadcasts in the Customer.IO dashboard
2. Configure your API credentials in the `.env` file
3. Test with your actual message IDs and customer data
4. Implement proper error handling in your production code

For data management operations (customer creation, event tracking), use the Data Pipelines API notebooks in the `pipelines_api/` directory.